In [2]:
# Modules used for handling and visualization of data
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import numpy as np

# Modules used for computer vision
import cv2

# Modules used for handling files
import os
from xml.dom import minidom
import xml.etree.ElementTree as ET
import json
import shutil

# Modules used for randomization
import random as r
import pprint

# Constants
TRAIN_DIR = "../data/atrw_detection_train/trainval"
TEST_DIR = "../data/atrw_detection_test/test/"
TRAIN_ANNOT_DIR = "../data/atrw_anno_detection_train(1)/Annotations"
TAGS = ["filename", "size", "width", "height", "depth", "object", "bndbox", "xmin", "xmax", "ymin", "ymax"]

In [128]:
# Convert the annotations from XML to TXT format.
class PASCAL2YOLO:
    
   # Initialize the variables for the class
   def __init__(self, obj_map : dict):
        self.obj_map = obj_map

   # Extraction of the bounding box dimensions by parsing XML.
   def __get_bounding_box_dims(self, id: str):
        annot = {"objects": []}
        
        # Parsing the XML annotations file. 
        xml_path = os.path.join(ANNOT_DIR, id+".xml")
        file = ET.parse(xml_path)
        
        # Traversal through the XML Tree.
        root=file.getroot()
        
        for i in root:
            # print(i.tag)
            if i.tag in TAGS:

               if i.tag == "size":

                  img_size = []
                  for img_dim in i:
                    img_size.append(int(img_dim.text))
                  annot[i.tag] = tuple(img_size)

               elif i.tag == "object":
                  tgr = {"name": i[0].text}
                  for bnd_dim in i[-1]:
                    tgr[bnd_dim.tag] = int(bnd_dim.text)
                  annot["objects"].append(tgr)

               else:
                  annot[i.tag] = i.text
        return annot

   def __convert_txt_format(self, id : str):
      annot = self.__get_bounding_box_dims(id)
      #pprint.pprint(annot)
      img_h, img_w, img_c = annot["size"]
      new_annotations = []

      for tgr in annot["objects"]:

         obj_id = self.obj_map[tgr["name"]]

         xmin = tgr["xmin"]
         xmax = tgr["xmax"]
         ymin = tgr["ymin"]
         ymax = tgr["ymax"]
         center_x = (xmin + xmax)/(2*img_w)
         center_y = (ymin + ymax)/(2*img_h)
         center_w = (xmax - xmin)/img_w
         center_h = (ymax - ymin)/img_h

         label = "{} {:.3f} {:.3f} {:.3f} {:.3f}".format(obj_id, center_x, center_y, 
                                                         center_w, center_h)

         new_annotations.append(label)
         
      return new_annotations
   
   def __write_annotations_to_txt(self, id : str):
      #print("Entered")
      new_annotations = self.__convert_txt_format(id)
      txt_path = os.path.join("../data/labels/", id + ".txt")

      fp = open(txt_path, "w")
      fp.write("\n".join(new_annotations))
      fp.close()
   
   def convert_all_annotations(self):

      all_annotations = os.listdir(ANNOT_DIR)
      all_ids = list(map(lambda p: p.replace(".xml", ""), all_annotations))

      for id in all_ids:
         self.__write_annotations_to_txt(id)
         
      print("Executed")

In [129]:
x = PASCAL2YOLO({"Tiger":0})

In [130]:
x.convert_all_annotations()

Executed


In [196]:
# Convert the annotations from JSON to TXT format.
class COCO2YOLO:

    # Initialize the variables for the class
    def __init__(self, obj_map : dict, annotations : list):
        self.obj_map = obj_map
        self.annotations = annotations
    
    def __get_dims(self, id : str):
        img_id = int(id)
        annot = {"objects": []}
        annot["filename"] = img_id

        for i in annotations["annotations"]:
            if i["image_id"] == img_id:
               tgr = {"name": "Tiger"}
               tgr["x"], tgr["y"], tgr["w"], tgr["h"] = i["bbox"]
               annot["objects"].append(tgr)
        
        return annot

    def __convert_txt_format(self, id : str):
      annot = self.__get_dims(id)
      #pprint.pprint(annot)
      img_h, img_w, img_c = (1920, 1080, 3)
      new_annotations = []

      for tgr in annot["objects"]:

         obj_id = self.obj_map[tgr["name"]]

         x = tgr["x"]
         y = tgr["y"]
         w = tgr["w"]
         h = tgr["h"]

         center_x = (x + (x + w))/(2*img_w)
         center_y = (y + (y + h))/(2*img_h)
         center_w = w/img_w
         center_h = h/img_h

         label = "{} {:.3f} {:.3f} {:.3f} {:.3f}".format(obj_id, center_x, center_y, 
                                                         center_w, center_h)

         new_annotations.append(label)
         
      return new_annotations

    def __write_annotations_to_txt(self, id : str):
      #print("Entered")
      new_annotations = self.__convert_txt_format(id)
      txt_path = os.path.join("../data/labels/test", id + ".txt")

      fp = open(txt_path, "w")
      fp.write("\n".join(new_annotations))
      fp.close()
    
    
    def convert_all_annotations(self):

      all_annotations = os.listdir(TEST_DIR)
      all_ids = list(map(lambda p: p.replace(".jpg", ""), all_annotations))

      for id in all_ids:
         self.__write_annotations_to_txt(id)
         
      print("Executed")

In [197]:
annotations = json.load(open("../data/atrw_anno_detection_test/test_labels.json", "r"))

In [198]:
x = COCO2YOLO({"Tiger":0}, annotations["annotations"])

In [199]:
x.convert_all_annotations()

Executed
